In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
import sys
from functools import reduce
import socket
import struct
import ipaddress


df1 = pd.read_csv('/content/arpcode (1).csv')

df = df1.iloc[0:2461,0:8]

df=df.fillna(0)
df.head(10)




,Time,Source,Destination,Protocol,Length,Info,next,diff
0,0.000000,PCSSystemtec_3a:59:87,7e:57:ab:7a:2b:f5,1.0,42.0,0.0,0.000000,0.000000
1,2.001191,PCSSystemtec_3a:59:87,7e:57:ab:7a:2b:f5,1.0,42.0,0.0,0.000000,2.001191
2,4.002254,PCSSystemtec_3a:59:87,7e:57:ab:7a:2b:f5,1.0,42.0,0.0,2.001191,2.001063
3,6.002604,PCSSystemtec_3a:59:87,7e:57:ab:7a:2b:f5,1.0,42.0,0.0,4.002254,2.000350
4,8.003296,PCSSystemtec_3a:59:87,7e:57:ab:7a:2b:f5,1.0,42.0,0.0,6.002604,2.000692
5,9.029871,7e:57:ab:7a:2b:f5,Broadcast,1.0,60.0,0.0,8.003296,1.026575
6,10.003960,PCSSystemtec_3a:59:87,7e:57:ab:7a:2b:f5,1.0,42.0,0.0,9.029871,0.974089
7,12.004801,PCSSystemtec_3a:59:87,7e:57:ab:7a:2b:f5,1.0,42.0,0.0,10.003960,2.000841
8,14.006328,PCSSystemtec_3a:59:87,7e:57:ab:7a:2b:f5,1.0,42.0,0.0,12.004801,2.001528
9,16.007135,PCSSystemtec_3a:59:87,7e:57:ab:7a:2b:f5,1.0,42.0,0.0,14.006328,2.000806


In [ ]:
df["Protocol"]=pd.to_numeric(df["Protocol"])
df["Protocol"].astype(str)
df["Protocol"]=df["Protocol"].apply(float)
df['Protocol'].dtype

count = df['Info'].value_counts()
print(count)

Info
0.0    2195
1.0     266
Name: count, dtype: int64


In [ ]:

X = pd.concat([df["diff"], df["Protocol"]], axis=1)

y=df.iloc[:,5]
y = pd.Series(y)
df2 = pd.concat([X, y], axis=1)
X

,diff,Protocol
0,0.000000,1.0
1,2.001191,1.0
2,2.001063,1.0
3,2.000350,1.0
4,2.000692,1.0
...,...,...
2456,0.702182,1.0
2457,0.280625,0.0
2458,0.000061,0.0
2459,0.031643,0.0


In [ ]:
df2.corr()


,diff,Protocol,Info
diff,1.000000,0.554387,0.077936
Protocol,0.554387,1.000000,0.488338
Info,0.077936,0.488338,1.000000


In [ ]:
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_X = scaler.fit_transform(X)
scaled_X

array([[0.00000000e+00, 1.00000000e+00],
       [9.99831847e-01, 1.00000000e+00],
       [9.99768060e-01, 1.00000000e+00],
       ...,
       [3.06765702e-05, 0.00000000e+00],
       [1.58095751e-02, 0.00000000e+00],
       [1.65873311e-05, 0.00000000e+00]])

In [ ]:
scaled_X,y


(array([[0.00000000e+00, 1.00000000e+00],
        [9.99831847e-01, 1.00000000e+00],
        [9.99768060e-01, 1.00000000e+00],
        ...,
        [3.06765702e-05, 0.00000000e+00],
        [1.58095751e-02, 0.00000000e+00],
        [1.65873311e-05, 0.00000000e+00]]),
 0       0.0
 1       0.0
 2       0.0
 3       0.0
 4       0.0
        ... 
 2456    1.0
 2457    0.0
 2458    0.0
 2459    0.0
 2460    0.0
 Name: Info, Length: 2461, dtype: float64)

In [ ]:
# print the last 10 rows of y
y.tail(10)

2451    0.0
2452    0.0
2453    0.0
2454    0.0
2455    0.0
2456    1.0
2457    0.0
2458    0.0
2459    0.0
2460    0.0
Name: Info, dtype: float64

In [ ]:

def df_to_X_y(df, window_size):
  df_as_np = np.array(df)
  X=[]
  y=[]
  for i in range(len(df_as_np)-window_size):
    row=[r for r in df_as_np[i:i+window_size]]
    X.append(row)
    label=df_as_np[i+window_size][2]
    y.append(label)

  return np.array(X), np.array(y)

In [ ]:
win_length=11

batch_size=64
num_features=3

In [ ]:
df_scaled=pd.concat([ pd.DataFrame(scaled_X),pd.DataFrame(y)], axis=1)

X_mod, y_mod = df_to_X_y(df_scaled, win_length)
df_scaled


,0,1,Info
0,0.000000,1.0,0.0
1,0.999832,1.0,0.0
2,0.999768,1.0,0.0
3,0.999411,1.0,0.0
4,0.999582,1.0,0.0
...,...,...,...
2456,0.350823,1.0,1.0
2457,0.140206,0.0,0.0
2458,0.000031,0.0,0.0
2459,0.015810,0.0,0.0


In [ ]:

scaled_X_df=pd.DataFrame(scaled_X)

X_train = X_mod[0:1476]
X_valid = X_mod[1476:1968]
X_test = X_mod[1968:]

y_train = y_mod[0:1476]
y_valid = y_mod[1476:1968]
y_test = y_mod[1968:]



In [ ]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape

((1476, 11, 3), (1476,), (492, 11, 3), (492,), (482, 11, 3), (482,))

In [ ]:

model=tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(32,input_shape= (win_length, num_features), return_sequences=True))
#model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
model.add(tf.keras.layers.LSTM(16,activation="tanh", return_sequences=False))
model.add(tf.keras.layers.Dense(8,activation="tanh"))
#model.add(tf.keras.layers.LeakyReLU(alpha=0.5))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
 model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 11, 32)            4608      
                                                                 
 lstm_11 (LSTM)              (None, 16)                3136      
                                                                 
 dense_10 (Dense)            (None, 8)                 136       
                                                                 
 dropout_5 (Dropout)         (None, 8)                 0         
                                                                 
 dense_11 (Dense)            (None, 1)                 9         
                                                                 
Total params: 7889 (30.82 KB)
Trainable params: 7889 (30.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
y_test

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0.

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min')

model.compile( loss = "binary_crossentropy", optimizer=tf.optimizers.Adam(learning_rate=0.001), metrics = ["accuracy"]) #
class_weight = {0: 1.,
                1: 8.}
history= model.fit(X_train, y_train, epochs = 25, validation_data = (X_valid, y_valid), batch_size=2, shuffle = False, callbacks = [early_stopping])


Epoch 1/25
738/738 [==============================] - 15s 15ms/step - loss: 0.3291 - accuracy: 0.8848 - val_loss: 0.3745 - val_accuracy: 0.8902
Epoch 2/25
738/738 [==============================] - 9s 12ms/step - loss: 0.2745 - accuracy: 0.8984 - val_loss: 0.3067 - val_accuracy: 0.8902
Epoch 3/25
738/738 [==============================] - 9s 12ms/step - loss: 0.2128 - accuracy: 0.9322 - val_loss: 0.2986 - val_accuracy: 0.8496
Epoch 4/25
738/738 [==============================] - 10s 13ms/step - loss: 0.1933 - accuracy: 0.9438 - val_loss: 0.3033 - val_accuracy: 0.9228
Epoch 5/25
738/738 [==============================] - 10s 14ms/step - loss: 0.1731 - accuracy: 0.9478 - val_loss: 0.3023 - val_accuracy: 0.9268
Epoch 6/25
738/738 [==============================] - 9s 12ms/step - loss: 0.1684 - accuracy: 0.9472 - val_loss: 0.2865 - val_accuracy: 0.8963
Epoch 7/25
738/738 [==============================] - 10s 13ms/step - loss: 0.1615 - accuracy: 0.9499 - val_loss: 0.2879 - val_accuracy: 0.

In [ ]:
X_valid.dtype, y_valid.dtype

(dtype('float64'), dtype('float64'))

In [ ]:
X_test

array([[[4.47158474e-05, 0.00000000e+00, 0.00000000e+00],
        [4.93597205e-01, 0.00000000e+00, 0.00000000e+00],
        [2.81784781e-05, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.88356139e-05, 0.00000000e+00, 0.00000000e+00],
        [1.59709019e-02, 0.00000000e+00, 0.00000000e+00],
        [2.65796992e-05, 0.00000000e+00, 0.00000000e+00]],

       [[4.93597205e-01, 0.00000000e+00, 0.00000000e+00],
        [2.81784781e-05, 0.00000000e+00, 0.00000000e+00],
        [1.60543382e-02, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.59709019e-02, 0.00000000e+00, 0.00000000e+00],
        [2.65796992e-05, 0.00000000e+00, 0.00000000e+00],
        [4.95764300e-01, 0.00000000e+00, 0.00000000e+00]],

       [[2.81784781e-05, 0.00000000e+00, 0.00000000e+00],
        [1.60543382e-02, 0.00000000e+00, 0.00000000e+00],
        [4.64145500e-05, 0.00000000e+00, 0.00000000e+00],
        ...,
        [2.65796992e-05, 0.00000000e+00, 0.00000000e+00],
        [4.95764300e-01, 0.00

In [ ]:
predictions = model.predict(X_test)

16/16 [==============================] - 1s 5ms/step


In [ ]:
predictions

array([[0.07134598],
       [0.01164552],
       [0.05283703],
       [0.17117333],
       [0.26246345],
       [0.07128697],
       [0.00982283],
       [0.00548668],
       [0.00660586],
       [0.01349574],
       [0.06204625],
       [0.01124119],
       [0.04462316],
       [0.17937633],
       [0.27667496],
       [0.08944125],
       [0.01160903],
       [0.0055477 ],
       [0.00641371],
       [0.01205638],
       [0.05337764],
       [0.01079983],
       [0.0372917 ],
       [0.18714298],
       [0.29215217],
       [0.11142215],
       [0.01391185],
       [0.00559775],
       [0.00621714],
       [0.01081002],
       [0.0454943 ],
       [0.01036377],
       [0.0310893 ],
       [0.19598758],
       [0.31017873],
       [0.13722987],
       [0.0168218 ],
       [0.00564568],
       [0.00601577],
       [0.00973385],
       [0.03849234],
       [0.00991371],
       [0.02586631],
       [0.03238748],
       [0.09583658],
       [0.24941052],
       [0.03071216],
       [0.006

In [ ]:
predictions_binary = (predictions > 0.5).astype(int)
print(predictions_binary)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]


In [ ]:
y_test = y_test.astype(int)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions_binary) * 100
print(accuracy)

94.1908713692946
